In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

from torchsummaryX  import summary
import wandb
from tqdm import tqdm

### Create Fully Connected Network

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

### Set device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Hyperparameters

In [4]:
input_size = 28*28
num_classes = 10
learning_rate = 0.001
batch_size = 128
num_epochs = 10

### Load Data

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialize network

In [6]:
model = NeuralNetwork(input_size=input_size, num_classes=num_classes).to(device)

In [7]:
wandb.init(project="Learn_PyTorch", 
           name="2_Neural_Network",
           config={
                "epochs": num_epochs,
                "batch_size": batch_size,
                "learning_rate": learning_rate,
                "dataset": "MNIST",
              }
           )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nabang1010_work (sanslab_steam). Use `wandb login --relogin` to force relogin


### Loss and Optimizer

In [8]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


### Train Network

In [9]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # log
    if loader.dataset.train:
        wandb.log({"train_acc": float(num_correct)/float(num_samples)*100})
    else:
        wandb.log({"test_acc": float(num_correct)/float(num_samples)*100})

    model.train()

In [10]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data = data.to(device=device)
        target = target.to(device=device)
        # get to correct shape
        data = data.reshape(data.shape[0], -1)
        # forward
        scores = model(data)
        loss = criterion(scores, target)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()
    # log
    wandb.log({"loss": loss.item()})
    check_accuracy(train_loader, model)
    check_accuracy(test_loader, model)

wandb.finish()

Epoch 1/10


100%|██████████| 469/469 [00:07<00:00, 62.80it/s]


Checking accuracy on training data
Got 55374 / 60000 with accuracy 92.29
Checking accuracy on test data
Got 9254 / 10000 with accuracy 92.54
Epoch 2/10


100%|██████████| 469/469 [00:05<00:00, 85.01it/s]


Checking accuracy on training data
Got 56395 / 60000 with accuracy 93.99
Checking accuracy on test data
Got 9385 / 10000 with accuracy 93.85
Epoch 3/10


...

100%|██████████| 469/469 [00:05<00:00, 85.92it/s]


Checking accuracy on training data
Got 58616 / 60000 with accuracy 97.69
Checking accuracy on test data
Got 9679 / 10000 with accuracy 96.79
Epoch 10/10


100%|██████████| 469/469 [00:05<00:00, 85.80it/s]


Checking accuracy on training data
Got 58758 / 60000 with accuracy 97.93
Checking accuracy on test data
Got 9683 / 10000 with accuracy 96.83


loss,▇▇█▄▄▆▁▄▁▁
test_acc,▁▃▄▅▆▇▇▇██
train_acc,▁▃▄▅▆▆▇▇██
loss,0.0682
test_acc,96.83
train_acc,97.93


### Save weight

In [11]:
weights_fd_path = "C:/Users/NAB/nabang1010/Learn_PyTorch/Pytorch_CV/weights"

# save model
torch.save(model.state_dict(), weights_fd_path + "/2_Neural_Network.pth")

# save jit model
model_scripted = torch.jit.script(model) 
model_scripted.save(weights_fd_path + "/2_Neural_Network_jit.pth")
